## Importing important libraries

In [32]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import datetime
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
import warnings
warnings.filterwarnings('ignore')

## Function -1
#### For generating prediction value on raw data

In [25]:
def generating_prediction(train,input_sentence,Original_Sentence): 
    if input_sentence.endswith('>'):
        input_sentence = input_sentence[:-1]
    if Original_Sentence.endswith('>'):
        Original_Sentence = Original_Sentence[:-1]       
    train.iloc[0]['informal_text_inp']= str(train.iloc[0]['informal_text_inp'])+'>'
    train.iloc[0]['normal_text_inp']= str(train.iloc[0]['normal_text_inp'])+'>'
    tknizer_informal = Tokenizer(filters = '"#$%&()*+-/=@[\\]^_`{|}~\t\n', lower = False, char_level = True)
    tknizer_informal.fit_on_texts(train['informal_text_inp'].values)
    tknizer_normal = Tokenizer(filters = '"#$%&()*+-/=@[\\]^_`{|}~\t\n', lower = False, char_level = True)
    tknizer_normal.fit_on_texts(train['normal_text_inp'].values)

    vocab_size_informal=len(tknizer_informal.word_index.keys())
    vocab_size_normal=len(tknizer_normal.word_index.keys())
    UNITS = 200
    MAX_LEN = 200
    model = tf.keras.models.load_model('Model_concat', custom_objects={"loss_function": loss_function})

    inputs = [tknizer_informal.word_index.get(i, 0) for i in input_sentence]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen = MAX_LEN, padding = 'post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = tf.zeros([1, UNITS]), tf.zeros([1, UNITS])
    enc_out, state_h, state_c = model.encoder([inputs, hidden])
    dec_hidden = [state_h, state_c]
    dec_input = tf.expand_dims([tknizer_normal.word_index['<']], 0)
    for t in range(MAX_LEN):
        output, state_h, state_c = model.decoder.timestepdecoder([dec_input, enc_out, state_h, state_c])
        predicted_id = tf.argmax(output[0]).numpy()
        if tknizer_normal.index_word.get(predicted_id, '') == '>':
            break
        else:
            result += tknizer_normal.index_word.get(predicted_id, '')
            dec_input = tf.expand_dims([predicted_id], 0)
    return result,Original_Sentence

In [29]:
import random
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

df=pd.read_csv('/content/with_augmentation_2.csv')
train, validation = train_test_split(df, test_size=0.05, random_state = 859)
train, test = train_test_split(train, test_size=0.05, random_state = 859)
data_for_prediction = test['informal_text_inp'].iloc[0]
Original_Sentence   = test['normal_text_out'].iloc[0]
pred,Original=generating_prediction(train,data_for_prediction,Original_Sentence)
print('Input  Sentence is    : ',data_for_prediction)
print('Predicted sentence is : ',pred)
print('Original Sentence is  : ',Original)

Input  Sentence is    :  <Watche Just Married. Haha.>
Predicted sentence is :  Watch Just Marail.
Original Sentence is  :  Watch Just Married. Haha.


## Function 2
#### For getting BLEU SCORE

In [33]:
def generating_prediction(train,input_sentence,Original_Sentence): 
    if input_sentence.endswith('>'):
        input_sentence = input_sentence[:-1]
    if Original_Sentence.endswith('>'):
        Original_Sentence = Original_Sentence[:-1]       
    train.iloc[0]['informal_text_inp']= str(train.iloc[0]['informal_text_inp'])+'>'
    train.iloc[0]['normal_text_inp']= str(train.iloc[0]['normal_text_inp'])+'>'
    tknizer_informal = Tokenizer(filters = '"#$%&()*+-/=@[\\]^_`{|}~\t\n', lower = False, char_level = True)
    tknizer_informal.fit_on_texts(train['informal_text_inp'].values)
    tknizer_normal = Tokenizer(filters = '"#$%&()*+-/=@[\\]^_`{|}~\t\n', lower = False, char_level = True)
    tknizer_normal.fit_on_texts(train['normal_text_inp'].values)

    vocab_size_informal=len(tknizer_informal.word_index.keys())
    vocab_size_normal=len(tknizer_normal.word_index.keys())
    UNITS = 200
    MAX_LEN = 200
    model = tf.keras.models.load_model('Model_concat', custom_objects={"loss_function": loss_function})

    inputs = [tknizer_informal.word_index.get(i, 0) for i in input_sentence]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen = MAX_LEN, padding = 'post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = tf.zeros([1, UNITS]), tf.zeros([1, UNITS])
    enc_out, state_h, state_c = model.encoder([inputs, hidden])
    dec_hidden = [state_h, state_c]
    dec_input = tf.expand_dims([tknizer_normal.word_index['<']], 0)
    for t in range(MAX_LEN):
        output, state_h, state_c = model.decoder.timestepdecoder([dec_input, enc_out, state_h, state_c])
        predicted_id = tf.argmax(output[0]).numpy()
        if tknizer_normal.index_word.get(predicted_id, '') == '>':
            break
        else:
            result += tknizer_normal.index_word.get(predicted_id, '')
            dec_input = tf.expand_dims([predicted_id], 0)
    result=result.split()       
    Original_Sentence=Original_Sentence.split() 
    score=bleu.sentence_bleu([Original_Sentence], result)      
    return score

In [34]:
import random
import nltk.translate.bleu_score as bleu
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

df=pd.read_csv('/content/with_augmentation_2.csv')
train, validation = train_test_split(df, test_size=0.05, random_state = 859)
train, test = train_test_split(train, test_size=0.05, random_state = 859)
data_for_prediction = test['informal_text_inp'].iloc[0]
Original_Sentence   = test['normal_text_out'].iloc[0]
score=generating_prediction(train,data_for_prediction,Original_Sentence)
print('Bleu score of predicted sentence is: ',score)

Bleu score of predicted sentence is:  0.5444460596606694
